# 11장

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

he_avg_init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init);

In [30]:
keras.layers.Dense(10, kernel_initializer='he_normal')
keras.layers.LeakyReLU(alpha=0.2)

keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal');

In [36]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(momentum=0.99), # momentum: 기본 0.99 미니배치가 작을수록 소수점 뒤에 9를 넣어 1에 가깝게 만듦
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_4 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_14 (Dense)            (None, 100)               30000     
                                                                 
 batch_normalization_5 (Batc  (None, 100)             

In [59]:
optimizer = keras.optimizers.SGD(clipvalue=1.0, clipnorm=1.0)
# clipvalue=1.0: loss의 모든 편미분 값을 -1.0 ~ 1.0으로 잘라냄.
# clipnorm=1.0: 해당 값 기준으로 정규화
# 두 인자 모두 기입 시 norm을 먼저 적용
model.compile(loss='mse', optimizer=optimizer)

In [60]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [61]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5488 - accuracy: 0.8273 - val_loss: 0.3828 - val_accuracy: 0.8697
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3566 - accuracy: 0.8775 - val_loss: 0.3307 - val_accuracy: 0.8832
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3202 - accuracy: 0.8893 - val_loss: 0.3140 - val_accuracy: 0.8929
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3004 - accuracy: 0.8967 - val_loss: 0.2933 - val_accuracy: 0.9013
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2870 - accuracy: 0.9007 - val_loss: 0.2798 - val_accuracy: 0.9048
Epoch 6/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2768 - accuracy: 0.9043 - val_loss: 0.2730 - val_accuracy: 0.9051
Epoch 7/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2683 - accuracy: 0.9086 - val_loss: 0.2675 - val_accuracy:

In [62]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 41ms/step - loss: 0.4940 - accuracy: 0.8200 - val_loss: 0.4351 - val_accuracy: 0.8641
Epoch 2/20
7/7 [==============================] - 0s 36ms/step - loss: 0.3594 - accuracy: 0.9450 - val_loss: 0.3553 - val_accuracy: 0.9016
Epoch 3/20
7/7 [==============================] - 0s 28ms/step - loss: 0.2905 - accuracy: 0.9400 - val_loss: 0.3010 - val_accuracy: 0.9381
Epoch 4/20
7/7 [==============================] - 0s 27ms/step - loss: 0.2438 - accuracy: 0.9750 - val_loss: 0.2590 - val_accuracy: 0.9604
Epoch 5/20
7/7 [==============================] - 0s 24ms/step - loss: 0.2081 - accuracy: 0.9850 - val_loss: 0.2305 - val_accuracy: 0.9675
Epoch 6/20
7/7 [==============================] - 0s 21ms/step - loss: 0.1832 - accuracy: 0.9850 - val_loss: 0.2067 - val_accuracy: 0.9706
Epoch 7/20
7/7 [==============================] - 0s 21ms/step - loss: 0.1623 - accuracy: 0.9850 - val_loss: 0.1894 - val_accuracy: 0.9736
Epoch 8/20
7/7 [===========

In [63]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # output 제외 전체 layer 반환
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

In [64]:
model_A_clone = keras.models.clone_model(model_A)   # 모델 구조 복사, 가중치는 복제하지 않음
model_A_clone.set_weights(model_A.get_weights())    # 가중치 복제

In [65]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False # 출력층 제외 가중치 동결
# 층을 동결하거나 동결 해제 후 새로 컴파일 필수
model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))